# FaceGreeter Camera Setup

This notebook sets up a Flask server in Google Colab to serve the FaceGreeter backend, cloning the `ai-greeter-core` repository, installing dependencies, and exposing the API via ngrok. It aims to resolve kernel crashes and connection issues (April 21) by using threading for Flask. The goal is to provide a stable backend for `https://facegreeter.vercel.app`, with plans to add video processing and V380 camera integration (April 17).

## Prerequisites
- Google Colab environment with internet access.
- `NGROK_AUTHTOKEN` stored in Colab Secrets.
- GitHub repository: `https://github.com/ATEMMETA/ai-greeter-core`.

## Steps
1. Install dependencies (`flask`, `flask-cors`, `pyngrok`, `python-dotenv`).
2. Clone the `ai-greeter-core` repository.
3. Set up `.env` with `NGROK_AUTHTOKEN` and Vercel URLs.
4. Run Flask server with ngrok using threading.
5. Monitor resources and test `/hello` endpoint.

**Goal**: Stable Flask API accessible via ngrok, ready for video processing.

In [ ]:
# Install dependencies and monitor resources
import os
import psutil
import shutil
import subprocess

# Log initial resource usage
print(f'Initial RAM: {psutil.virtual_memory().used / 1024**3:.2f} GB / {psutil.virtual_memory().total / 1024**3:.2f} GB')
print(f'Initial CPU: {psutil.cpu_percent()}%')

# Purge pip cache
!pip cache purge

# Install dependencies
!pip install --no-cache-dir flask==3.0.3 flask-cors==5.0.0 pyngrok==7.1.6 python-dotenv==1.0.1

# Verify installed packages
!pip list | grep -E 'flask|pyngrok|python-dotenv'

# Log resource usage after install
print(f'Post-install RAM: {psutil.virtual_memory().used / 1024**3:.2f} GB / {psutil.virtual_memory().total / 1024**3:.2f} GB')
print(f'Post-install CPU: {psutil.cpu_percent()}%')

# Clear existing ai-greeter-core directory
if os.path.exists('/content/ai-greeter-core'):
    shutil.rmtree('/content/ai-greeter-core')
os.makedirs('/content/ai-greeter-core', exist_ok=True)

# Clone repository
repo_url = 'https://github.com/ATEMMETA/ai-greeter-core.git'
clone_command = f'git clone {repo_url} /content/ai-greeter-core'
print(f'Cloning repository: {clone_command}')
clone_process = subprocess.run(clone_command, shell=True, capture_output=True, text=True)

# Check for clone success
required_files = ['/content/ai-greeter-core/requirements.txt', '/content/ai-greeter-core/combined_app.py']
files_missing = [f for f in required_files if not os.path.exists(f)]
if clone_process.returncode != 0 or files_missing:
    print(f'Git clone failed or files missing. Error: {clone_process.stderr}')
    print(f'Missing files: {files_missing}')
    raise FileNotFoundError('Required files missing after git clone.')
else:
    print('Git clone succeeded. Files found:', required_files)

print('Dependencies installed and repository cloned. Proceed to the next cell to set up the environment.')

In [ ]:
# Set up environment
from google.colab import userdata
import os

# Get NGROK_AUTHTOKEN from Colab Secrets
ngrok_token = userdata.get('NGROK_AUTHTOKEN')
if not ngrok_token:
    raise ValueError('NGROK_AUTHTOKEN not found in Colab Secrets.')

# Create .env file
env_path = '/content/ai-greeter-core/.env'
with open(env_path, 'w') as f:
    f.write(f'NGROK_AUTHTOKEN={ngrok_token}\n')
    f.write('VERCEL_URL=https://facegreeter.vercel.app\n')
    f.write('VERCEL_API_BASE_URL=https://facegreeter.vercel.app\n')

print(f'.env file created at {env_path} with NGROK_AUTHTOKEN and Vercel URLs.')
print('Environment setup complete. Proceed to the next cell to run the Flask server.')

In [ ]:
# Run Flask with ngrok
from pyngrok import ngrok
import os
import psutil
import threading
import time
from dotenv import load_dotenv

# Kill existing ngrok and Flask processes
os.system('pkill ngrok')
os.system('pkill python')
os.system('fuser -k 8000/tcp')

# Verify combined_app.py exists
if not os.path.exists('/content/ai-greeter-core/combined_app.py'):
    raise FileNotFoundError('combined_app.py not found. Ensure git clone succeeded or fallback files were created.')

# Load environment variables
load_dotenv('/content/ai-greeter-core/.env')
ngrok_token = os.getenv('NGROK_AUTHTOKEN')
if not ngrok_token:
    raise ValueError('NGROK_AUTHTOKEN not found in .env file.')

# Set ngrok auth token
ngrok.set_auth_token(ngrok_token)

# Log resource usage before Flask
print(f'Pre-Flask RAM: {psutil.virtual_memory().used / 1024**3:.2f} GB / {psutil.virtual_memory().total / 1024**3:.2f} GB')
print(f'Pre-Flask CPU: {psutil.cpu_percent()}%')

# Run Flask from combined_app.py in a thread
def run_flask():
    print('Starting Flask server on http://0.0.0.0:8000')
    os.system('python /content/ai-greeter-core/combined_app.py')

print('Starting Flask server...')
flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

# Wait for Flask to start
time.sleep(5)

# Start ngrok tunnel
try:
    public_url = ngrok.connect(8000).public_url
    print(f'Flask API available at: {public_url}')
except Exception as e:
    print(f'Ngrok failed to start: {str(e)}')
    os.system('pkill python')
    os.system('pkill ngrok')
    os.system('fuser -k 8000/tcp')
    raise

# Check running processes and port
os.system('ps aux | grep python')
os.system('netstat -tuln | grep 8000')

# Log resource usage after Flask
print(f'Post-Flask RAM: {psutil.virtual_memory().used / 1024**3:.2f} GB / {psutil.virtual_memory().total / 1024**3:.2f} GB')
print(f'Post-Flask CPU: {psutil.cpu_percent()}%')

# Keep the notebook running
print(f'Flask server running. Test the API at {public_url}/hello.')
try:
    flask_thread.join()
except KeyboardInterrupt:
    print('Stopping Flask server...')
    os.system('pkill ngrok')
    os.system('pkill python')
    os.system('fuser -k 8000/tcp')